## Challenge 1-2

In [1]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

# Replace with a valid key
training_key = ""
prediction_key = ""

trainer = training_api.TrainingApi(training_key)

# Create a new project
print ("Creating project...")
project = trainer.create_project("aiworkshop")

ImportError: No module named 'azure.cognitiveservices'

In [ ]:
# Make two tags in the new project
hardshell_tag = trainer.create_tag(project.id, "Hardshell jackets")
insulated_tag = trainer.create_tag(project.id, "Insulated jackets")

In [ ]:
# Upload images to the project

import os
import time

hardshell_dir = "gear_images\\hardshell_jackets"
for image in os.listdir(os.fsencode(hardshell_dir)):
    with open(hardshell_dir + "\\" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data.read(), [ hardshell_tag.id ])

insulated_dir = "gear_images\\insulated_jackets"
for image in os.listdir(os.fsencode(insulated_dir)):
    with open(insulated_dir + "\\" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data.read(), [ insulated_tag.id ])

In [ ]:
 Train the project

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status == "Training"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

# Open the sample image and get back the prediction results.
with open("gear_images\\hardshell_jackets\\112128.jpeg", mode="rb") as test_data:
    results = predictor.predict_image(project.id, test_data.read(), iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag + ": {0:.2f}%".format(prediction.probability * 100))